In [49]:
# ファイル読み込み
#MDtrajのtrajオブジェクトを作成する
import mdtraj
traj=mdtraj.load("si_2/si_traj.xyz", top="si_2/si_traj.pdb")

import nglview as nv
view=nv.show_mdtraj(traj,gui=True)
view.parameters =dict(
                        camera_type="orthographic",
                        backgraound_color="black",
                        clip_dist=0
)
view.clear_representations()
view.add_representation("ball+stick")
#view.add_representation("spacefill",selection=[i for i in range(n_atoms,n_total_atoms)],opacity=0.1)
view.add_unitcell()
view.update_unitcell()
view

NGLWidget(max_frame=299)

In [14]:
#====================
# 構造データの読み込み: mdtrajデータを直接読み込み
#====================
# eq_pbc.trrと，eq.pdbの二つのファイルを持ってくる必要がある．
#
#
def load_traj(trr_filename,pdb_filename, NUM_MOL=int(108), PLOT=False):
    #
    #########  input #############
    #  directory:: eq.pdbとeq_pbc.trrが入っているディレクトリ
    #  NUM_MOL:: 分子数の指定(trajファイルからatom数は指定できるが分子数を取り出せるかわからないので．)
    #  #ベンゼン108/シクロヘキサン108だったのでデフォルトを108にしておく
    #  
    #
 
    import os
    is_file = os.path.isfile(trr_filename)
    if not is_file:
        print("ERROR :: can not find eq_pbc.trr")
        return 1

    is_file = os.path.isfile(pdb_filename)
    if not is_file:
        print("ERROR :: can not find eq.pdb")
        return 1
    #
    print("")
    print(" success:: load eq_pbc.trr/eq.pdb")
    print("")
    
    #
    #diff_config=np.load("8ben_STR/original_structure.npy")
    #diff_config=np.load("benzene_MD_output2/original_structure.npy")
    import mdtraj
    import numpy as np
    traj=mdtraj.load(trr_filename, top=pdb_filename)
    
    from  ase.io.espresso import  read_espresso_in
    
    # トポロジーの作成．一応ここからNUM_MOLを取り出せるが．．．
    # Residue Sequence record (generally 1-based, but depends on topology)
    table, bonds =traj.topology.to_dataframe()
    resseq=np.array(table["resSeq"])

    # 種々のデータをloadする．
    NUM_ATOM    =traj.n_atoms #原子数
    NUM_CONFIG  =traj.n_frames #フレーム数
    VOLUME      =traj.unitcell_volumes # 0番目のconfigurationの体積[nm^3]
    ATOM_PER_MOL=int(NUM_ATOM/NUM_MOL)  # 1分子あたりの原子数
    TIMESTEP    =traj.timestep # in psec 
    UNITCELL_VECTORS = traj.unitcell_vectors  # 最初のconfigのvectorを出力3*3行列

    #============
    print("##=================")
    print("NUM_ATOM             ：：", NUM_ATOM)
    print("NUM_CONFIG           ：：", NUM_CONFIG)
    print("NUM_MOL              ：：", NUM_MOL)
    print("VOLUME[nm^3]         ：：", VOLUME)
#    print("セルパラメータ(cubic)[nm]     ：：", np.cbrt(VOLUME))
    print("ATOM_PER_MOL         ：：", ATOM_PER_MOL)
    print("TIMESTEP[ps]         ：：", TIMESTEP)
    print("unitcell vectors[nm] ：：", UNITCELL_VECTORS)


    # ============================
    # 読み込んだ構造データを図示する．
    # ============================
    #
    if PLOT:
        import nglview as nv
        view=nv.show_mdtraj(traj,gui=True)
        view.parameters=dict(
            #density=1.02 #pyrimidine
            camera_type="orthographic",
            background_color="black",
            clip_dist=0)
        view.clear_representations()
        view.add_representation("ball+stick")
        view.add_unitcell()
        view.update_unitcell()
        view

    #
    return traj, NUM_ATOM, NUM_CONFIG,NUM_MOL, VOLUME, ATOM_PER_MOL, UNITCELL_VECTORS, TIMESTEP, resseq



In [15]:
traj, NUM_ATOM, NUM_CONFIG,NUM_MOL, VOLUME, ATOM_PER_MOL, UNITCELL_VECTORS, TIMESTEP, resseq = load_traj("si_2/si_traj.xyz", "si_2/si_traj.pdb", NUM_MOL=24, PLOT=False)


 success:: load eq_pbc.trr/eq.pdb

##=================
NUM_ATOM             ：： 8
NUM_CONFIG           ：： 300
NUM_MOL              ：： 24
VOLUME[nm^3]         ：： None
ATOM_PER_MOL         ：： 0
TIMESTEP[ps]         ：： 1
unitcell vectors[nm] ：： None


In [112]:
import ase.io
test=ase.io.read("si_2/si_traj.xyz", index=":")
for i in range(len(test)):
    test[i].set_cell([5.6308,5.6308,5.6308]) #Ang
#for i in range(len(test)):
#    test[i].set_pbc((True,True,True))
#
#for i in range(len(test)):
#    test[i].set_pbc((False,False,False))
#

import numpy as np
for i in np.arange(1,len(test)):
    coord=test[i].get_positions()
    coord_before=test[i-1].get_positions()
    # jumpする場合
    if np.any(np.abs(coord_before-coord)>2.8):
        print("step :: ", i)
        # jumpする場合には格子定数を追加する．
        tmp1=np.where(coord-coord_before>2.8 , coord, coord+5.6308)
        tmp2=np.where(coord-coord_before<-2.8, tmp1, tmp1-5.6308)
        test[i].set_positions(tmp2)

print("fin nojump")
import numpy as np
for i in np.arange(1,len(test)):
    coord=test[i].get_positions()
    coord_before=test[i-1].get_positions()
    # jumpする場合
    if np.any(np.abs(coord_before-coord)>2.8):
        print("step :: ", i)


print(test[0].get_atomic_numbers())
print(test[0].get_cell())
print(test[0].pbc)
print(test[0].get_center_of_mass())

#from ase.visualize import view
#view(test)

print(test[1].cell)

ase.io.write("si_2/si_test.xyz",images=test) #,format="xyz")


# ファイル読み込み
#MDtrajのtrajオブジェクトを作成する
import mdtraj
cells=np.array([[5.6308,0,0],[0,5.6308,0],[0,0,5.6308]])
length=np.array([5.6308,5.6308,5.6308])/10.0 # ang to nm
angle=np.array([90,90,90])
lengths_array=np.tile(length,(len(test),1))
angle_array=np.tile(angle,(len(test),1))
print("angle_array.shape", np.shape(angle_array))

traj0=mdtraj.load("si_2/si_test.xyz", top="si_2/si_traj.pdb") #, unitcell_lengths=np.tile(length,(len(test),1))) #,unitcell_vectors=[cells for i in range(len(test))])

xyz=traj0.xyz
top=traj0.top

traj=mdtraj.Trajectory(xyz, top, unitcell_lengths=lengths_array, unitcell_angles=angle_array) #, unitcell_lengths=np.tile(length,(len(test),1))) #,unitcell_vectors=[cells for i in range(len(test))])

# trajをsave
traj.save_trr("si_2/si_nojump_traj.trr")

#print("mdtraj unitcell", traj.unitcell_vectors)
#print(traj.unitcell_lengths)
import nglview as nv
view=nv.show_mdtraj(traj,gui=True)
view.parameters =dict(
                        camera_type="orthographic",
                        backgraound_color="black",
                        clip_dist=0
)
view.clear_representations()
view.add_representation("ball+stick")
#view.add_representation("spacefill",selection=[i for i in range(n_atoms,n_total_atoms)],opacity=0.1)
view.add_unitcell()
view.update_unitcell()
view

step ::  7
step ::  8
step ::  9
step ::  10
step ::  11
step ::  12
step ::  13
step ::  14
step ::  15
step ::  16
step ::  17
step ::  18
step ::  19
step ::  20
step ::  21
step ::  22
step ::  23
step ::  24
step ::  25
step ::  26
step ::  27
step ::  28
step ::  29
step ::  30
step ::  31
step ::  32
step ::  33
step ::  34
step ::  35
step ::  36
step ::  37
step ::  38
step ::  39
step ::  40
step ::  41
step ::  42
step ::  43
step ::  44
step ::  45
step ::  46
step ::  47
step ::  48
step ::  49
step ::  50
step ::  51
step ::  52
step ::  53
step ::  54
step ::  55
step ::  56
step ::  57
step ::  58
step ::  59
step ::  60
step ::  61
step ::  62
step ::  63
step ::  64
step ::  65
step ::  66
step ::  67
step ::  68
step ::  69
step ::  70
step ::  71
step ::  76
step ::  77
step ::  78
step ::  79
step ::  80
step ::  81
step ::  82
step ::  83
step ::  84
step ::  85
step ::  86
step ::  87
step ::  88
step ::  89
step ::  90
step ::  91
step ::  92
step ::  93
step ::

NGLWidget(max_frame=299)

In [ ]:
#
from ase import Atoms # MIC計算用 

    #
    # ATOMのリストはNUM_MOL個必要．
    atom_id= ['Au'] * CM_array.shape[0]
    #
    #  
    CM = Atoms(atom_id,
             positions=CM_array,        
             cell= UNITCELL_VECTORS,   
             pbc=[1, 1, 1]) #pbcは周期境界条件．これをxyz軸にかけることを表している．
             

In [59]:
# Coordinates are in Angstroms.
# The length of a timestep can be set by passing the dt argument, it’s assumed to be in ps (default: 1 ps).

import MDAnalysis
# 
#         jbcfrom MDAnalysis.tests.datafiles import PSF,DCD  # test trajectory
# hamonirc 
u = MDAnalysis.Universe("si_2/si_traj.xyz","si_2/si_traj.pdb")


import numpy.linalg
test=MDAnalysis.coordinates.XYZ.XYZReader("si_2/si_traj.xyz")

u.atoms.write('si_2/out.gro', reindex=False)


traj=mdtraj.load("si_2/out.gro", top="si_2/si_traj.pdb")
import nglview as nv
view=nv.show_mdtraj(traj,gui=True)
view.parameters =dict(
                        camera_type="orthographic",
                        backgraound_color="black",
                        clip_dist=0
)
view.clear_representations()
view.add_representation("ball+stick")
#view.add_representation("spacefill",selection=[i for i in range(n_atoms,n_total_atoms)],opacity=0.1)
view.add_unitcell()
view.update_unitcell()
view



NGLWidget()

In [11]:
print(u)
print(len(u.trajectory)) # frame数

<Universe with 8 atoms>
1


In [ ]:
import MDAnalysis.visualization.streamlines
generate_streamlines_3d()


In [108]:
#Trajectoryをmdtrajで変換するために．


# まずはxyzファイルをload. しかしmdtrajはaseがsupportするextended xyzをsupportしておらず，
# xyzから格子定数を読み込むことができない．そこで一旦traを読み込んだ後に，格子定数を追加した別のtrajectoryを生成する．
length=np.array([5.6308,5.6308,5.6308])/10.0 # ang to nm
angle=np.array([90,90,90])
lengths_array=np.tile(length,(len(test),1))
traj0=mdtraj.load("si_2/si_traj.xyz", top="si_2/si_traj.pdb") #, unitcell_lengths=np.tile(length,(len(test),1))) #,unitcell_vectors=[cells for i in range(len(test))])
# xyz,topを取得
xyz=traj0.xyz
top=traj0.top
traj=mdtraj.Trajectory(xyz, top, unitcell_lengths=lengths_array, unitcell_angles=angle_array) #, unitcell_lengths=np.tile(length,(len(test),1))) #,unitcell_vectors=[cells for i in range(len(test))])

# trajをsave
traj.save_trr("si_2/si_traj.trr")


#print("mdtraj unitcell", traj.unitcell_vectors)
#print(traj.unitcell_lengths)
import nglview as nv
view=nv.show_mdtraj(traj,gui=True)
view.parameters =dict(
                        camera_type="orthographic",
                        backgraound_color="black",
                        clip_dist=0
)
view.clear_representations()
view.add_representation("ball+stick")
#view.add_representation("spacefill",selection=[i for i in range(n_atoms,n_total_atoms)],opacity=0.1)
view.add_unitcell()
view.update_unitcell()
view


NGLWidget(max_frame=299)

In [114]:

#Trajectoryの表示
import sys
import mdtraj

# ParmEd Imports
from parmed import load_file
from parmed.openmm.reporters import NetCDFReporter
from parmed import unit as u

#analysis
import os  
import subprocess
from subprocess import PIPE

!echo "System" > ./inputs/anal.txt 


# -f [<.xtc/.trr/...>] (traj.xtc)
# Trajectory: xtc trr cpt gro g96 pdb tng
# -s [<.tpr/.gro/...>] (topol.tpr) (Optional)
# Structure+mass(db): tpr gro g96 pdb brk ent
commands = "gmx trjconv -s si_2/si_traj.pdb -f si_2/si_nojump_traj.trr -dump 0 -o si_2/si_trajconv.pdb < ./inputs/anal.txt"
proc = subprocess.run(commands, shell=True, stdout=PIPE, stderr=PIPE,encoding='utf-8')
output = proc.stdout
commands = "gmx trjconv -s si_2/si_traj.pdb -f si_2/si_nojump_traj.trr -pbc nojump -o si_2/si_pbc.trr < ./inputs/anal.txt" #-force は力を読み書きするのでとりあえずいらない．
proc = subprocess.run(commands, shell=True, stdout=PIPE, stderr=PIPE,encoding='utf-8')
output = proc.stdout

traj=mdtraj.load("si_2/si_pbc.trr") #, top="si_2/si_trajconv.pdb")
view=nv.show_mdtraj(traj,gui=True)
view.parameters =dict(
                        camera_type="orthographic",
                        backgraound_color="black",
                        clip_dist=0
)
view.clear_representations()
view.add_representation("ball+stick")
#view.add_representation("spacefill",selection=[i for i in range(n_atoms,n_total_atoms)],opacity=0.1)
view.add_unitcell()
view.update_unitcell()
view

OSError: The topology is loaded by filename extension, and the detected ".trr" format is not supported. Supported topology formats include ".pdb", ".pdb.gz", ".h5", ".lh5", ".prmtop", ".parm7", ".prm7", ".psf", ".mol2", ".hoomdxml", ".gro", ".arc", ".hdf5" and ".gsd".